In [28]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
import pandas as pd
pd.set_option('display.max_rows', None)
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.formula.api import mixedlm
from statsmodels.tools.eval_measures import rmse
import statsmodels
import statsmodels.api as sm
import numpy as np
from patsy.contrasts import Sum

In [29]:
dataset = pd.read_csv('/Users/zilianghong/Documents/GitHub/IPMN-Tissue-Analysis/IPMN_tissue_data_with_cyst.csv')
dataset = dataset.drop(columns=['name','label'])
dataset.replace({'AHN': 1, 'CAD': 2,'EMC':3,'IU':4, 'MCA':5, 'MCF':6, 'NU':7, 'NYU':8}, inplace=True)

/var/folders/9h/n7423dz95lv3x1z3vnm52bqr0000gs/T/ipykernel_40142/3056495064.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace({'AHN': 1, 'CAD': 2,'EMC':3,'IU':4, 'MCA':5, 'MCF':6, 'NU':7, 'NYU':8}, inplace=True)


In [30]:
rename_dict = {'cyst':'Y','pancreas':'Y1','visceral fat':"X2",'subcutaneous fat':"X3",'muscle':"X1",'center':"SUBJ"}
dataset = dataset.rename(columns=rename_dict)

## Contrast
Is there anysignificant effect of centers on cyst volume? Without blocking

In [31]:
Y = dataset['Y']
Y1 = dataset['Y1']
X1 = dataset['X1']
X2 = dataset['X2']
X3 = dataset["X3"]
X5 = dataset['SUBJ']

mod= ols('Y1~ 1+ X1 * X2 * X3', data=dataset).fit()
print(mod.summary())

ypred = mod.predict(dataset[['X1','X2','X3']])
RMSE = rmse(Y, ypred)
print('RMSE:' + str(RMSE))

# Perform ANOVA
anova_table = sm.stats.anova_lm(mod, typ=1)  # Type 2 ANOVA
print(anova_table) 

                            OLS Regression Results                            
Dep. Variable:                     Y1   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     4.985
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           2.24e-05
Time:                        22:44:39   Log-Likelihood:                -1637.5
No. Observations:                 332   AIC:                             3291.
Df Residuals:                     324   BIC:                             3321.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     66.4385      8.301      8.003      0.0

Repeat experiment, blocking the effect of centers

In [32]:

mod2= ols('Y1~ 1+ X1 * X2 * X3 + C(SUBJ)', data=dataset).fit()
print(mod.summary())

ypred = mod2.predict(dataset[['X1','X2','X3','SUBJ']])
RMSE = rmse(Y, ypred)
print('RMSE:' + str(RMSE))

# Perform ANOVA
anova_table = sm.stats.anova_lm(mod2, typ=1)  # Type 2 ANOVA
print(anova_table)

                            OLS Regression Results                            
Dep. Variable:                     Y1   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     4.985
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           2.24e-05
Time:                        22:44:39   Log-Likelihood:                -1637.5
No. Observations:                 332   AIC:                             3291.
Df Residuals:                     324   BIC:                             3321.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     66.4385      8.301      8.003      0.0

In [56]:
import matplotlib.pyplot as plt
def show_dis(col):
    plt.figure(figsize=(8, 6))  
    plt.hist(col, bins=100, color='skyblue', edgecolor='black')  
    plt.title('Distribution', fontsize=16)  
    plt.xlabel('Value', fontsize=14)  
    plt.ylabel('Frequency', fontsize=14)  
    plt.grid(alpha=0.3)  
    plt.show()

In [57]:
# Set the number of bootstrap samples
n_bootstrap = 100  # Number of bootstrap datasets to generate

# Set the size of each bootstrap sample (same as the original dataset)
sample_size = len(dataset)

# Create a list to store bootstrap samples
bootstrap_samples = []

# Generate bootstrap samples
for i in range(n_bootstrap):
    bootstrap_sample = dataset.sample(n=sample_size, replace=True)
    bootstrap_samples.append(bootstrap_sample)
bootstrap_df = bootstrap_samples[0]
for i in range(1,len(bootstrap_samples)):
    bootstrap_df = pd.concat(bootstrap_samples, ignore_index=True)

In [77]:
temp_data = dataset.sample(n=10, replace=True)
Y = temp_data['Y']
Y1 = temp_data['Y1']
X1 = temp_data['X1']
X2 = temp_data['X2']
X3 = temp_data["X3"]
X5 = temp_data['SUBJ']

mod2= ols('Y1~ 1+ X1 * X2 * X3', data=temp_data).fit()
print(mod.summary())

ypred = mod2.predict(temp_data[['X1','X2','X3']])
RMSE = rmse(temp_data['Y1'], ypred)
print('RMSE:' + str(RMSE))

# Perform ANOVA
anova_table = sm.stats.anova_lm(mod2, typ=1)  # Type 2 ANOVA
print(anova_table)

                            OLS Regression Results                            
Dep. Variable:                     Y1   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     4.985
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           2.24e-05
Time:                        23:28:09   Log-Likelihood:                -1637.5
No. Observations:                 332   AIC:                             3291.
Df Residuals:                     324   BIC:                             3321.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     66.4385      8.301      8.003      0.0